# No stimulation Demo Experiment
This notebook showcases a demo timelapse experiment, where no stimulation is applied to the cells. The experiment runs as a timelapse, and the cells are imaged at regular intervals. In this case the feature extractor is set to the baseline feature extractors, which just extract label position and area. For tracking trackpy is used and stardist is used for segmentation. 

As this is a demo experiment, it runs on the demo hardware provided by micro-manager. The demo hardware is a simulated hardware that mimics the behavior of a real microscope. 

### Import required libraries

In [1]:
import os
import time

os.environ["QT_LOGGING_RULES"] = (
    "*.debug=false; *.warning=false"  # Fix to suppress PyQT warnings from napari-micromanager when running in a Jupyter notebook
)
os.environ["MICROMANAGER_PATH"] = "C:\\Program Files\\Micro-Manager-2.0"

from rtm_pymmcore.data_structures import Fov, Channel, StimTreatment
from rtm_pymmcore.utils import create_folders
from pprint import pprint
import pandas as pd
import numpy as np
import dataclasses
import random
import pymmcore_plus

from useq._mda_event import SLMImage

mmc = pymmcore_plus.CMMCorePlus()

### Experimental Settings

In [2]:
## Configuration options
N_FRAMES = 2

SLEEP_BEFORE_EXPERIMENT_START_in_H = 0
USE_AUTOFOCUS_EVENT = False

TIME_BETWEEN_TIMESTEPS = 2  # time in seconds between frames
TIME_PER_FOV = 1  # time in seconds per fov


## Storage path for the experiment
base_path = "C:\\Users\\Alex\\Ausbildung\\PhD_temp\\test_exp"
experiment_name = "exp_test"
path = os.path.join(base_path, experiment_name)

path_with_old_data_for_simulation = os.path.join(
    ".", "test_exp_data", "00_01_demo_imgs"
)  # path to the folder with old data for simulation

# Define Channels for which Images are taken. If no power is defined, the default power of the device will be used,
# for example, see the second channel "Cy5" below. The default power is set in the GUI
channels = []
channels.append(
    Channel(
        name="DAPI",
        exposure=150,
        group="Channel",
        power=2,
        device_name="LED",
        property_name="State",
    )
)
channels.append(Channel(name="Cy5", exposure=150))


# Condition mapping to FOVs. This is used to create a dataframe with the conditions and the FOVs.
condition = [
    "FGFR_high"
]  # Example of adding a condition to the dataframe. Stimulation will be repeated for each condition.
# condition = ["optoFGFR_high"] * 24 + ["optoFGFR"] * 24 # Example of adding multiple conditions to the dataframe. n repreats the amount of times the condition is repeated.

n_fovs_per_condition = 36  ## change this variable to the amount of fovs that you have per cell line. If only one cell line is set, this value will
# automatically set to total amount of fovs.

n_fovs_per_well = None  ## change this variable to the amount of fovs that you have per well. Set to None if you are not working with wellplate.


## Define the Tools that you are using for the experiment
from rtm_pymmcore.segmentation.stardist import SegmentatorStardist
from rtm_pymmcore.tracking.trackpy import TrackerTrackpy
from rtm_pymmcore.feature_extraction.simple_fe import SimpleFE

segmentators = [
    {
        "name": "labels",
        "class": SegmentatorStardist(),
        "use_channel": 0,
        "save_tracked": True,
    },
]
stimulator = None
feature_extractor = SimpleFE("labels")
tracker = TrackerTrackpy()


from rtm_pymmcore.img_processing_pip import ImageProcessingPipeline

pipeline = ImageProcessingPipeline(
    storage_path=path,
    segmentators=segmentators,
    feature_extractor=feature_extractor,
    tracker=tracker,
    stimulator=stimulator,
)

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\raw already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\tracks already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\particles already exists
Directory C:\Users\Alex\Ausbildung\PhD_temp\test_exp\exp_test\labels already exists


### Load device and set startup channel

In [3]:
mmc.loadSystemConfiguration("C:\\Program Files\Micro-Manager-2.0\\MMConfig_demo.cfg")
mmc.setConfig(groupName="System", configName="Startup")
mmc.setChannelGroup(channelGroup="Channel")

### GUI - Napari Micromanager

#### Load GUI

In [4]:
### Base GUI ###
from napari_micromanager import MainWindow
import napari

viewer = napari.Viewer()
mm_wdg = MainWindow(viewer)
viewer.window.add_dock_widget(mm_wdg)
data_mda_fovs = None

### Add MDA widget for FOV selection ###
from pymmcore_widgets.mda import MDAWidget

mdawidget = MDAWidget(mmcore=mmc)
viewer.window.add_dock_widget(mdawidget)

#### Functions to break and re-connect link with GUI if manually broken

The following functions can be used to manually interrupt to connection between the GUI and the running rtm-pymmcore script. However, normally you don't need to execute them. 

In [ ]:
### Break connection
# mm_wdg._core_link.cleanup()

In [ ]:
### Manually reconnect pymmcore with napari-micromanager
from napari_micromanager._core_link import CoreViewerLink

mm_wdg._core_link = CoreViewerLink(viewer, mmc)

### Map Experiment to FOVs

#### If FOVs already saved - Reload them from file

In [4]:
import json

file = os.path.join(path, "fovs.json")
with open(file, "r") as f:
    data_mda_fovs = json.load(f)

### Use FOVs to generate dataframe for acquisition

In [5]:
n_fovs_simultaneously = TIME_BETWEEN_TIMESTEPS // TIME_PER_FOV
timesteps = range(N_FRAMES)

start_time = 0
if data_mda_fovs is None:
    assert False, "No fovs selected. Please select fovs in the MDA widget"
dfs = []
fovs = []
for fov_index, fov in enumerate(data_mda_fovs):
    fov_object = Fov(fov_index)
    fovs.append(fov_object)
    fov_group = fov_index // n_fovs_simultaneously
    start_time = fov_group * TIME_BETWEEN_TIMESTEPS * len(timesteps)
    if len(condition) == 1:
        condition_fov = condition[0]
    else:
        condition = condition[fov_index // n_fovs_per_condition]
    for timestep in timesteps:
        row = {
            "fov_object": fov_object,
            "fov": fov_index,
            "fov_x": fov.get("x"),
            "fov_y": fov.get("y"),
            "fov_z": fov.get("z"),
            "fov_name": str(fov_index) if fov["name"] is None else fov.name,
            "timestep": timestep,
            "time": start_time + timestep * TIME_BETWEEN_TIMESTEPS,
            "cell_line": condition_fov,
            "channels": tuple(dataclasses.asdict(channel) for channel in channels),
            "fname": f"{str(fov_index).zfill(3)}_{str(timestep).zfill(5)}",
        }
        dfs.append(row)

df_acquire = pd.DataFrame(dfs)

print(f"Total Experiment Time: {df_acquire['time'].max()/3600}h")


df_acquire = df_acquire.dropna(axis=1, how="all")
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", True)
df_acquire = df_acquire.sort_values(by=["time", "fov"])
df_acquire

Total Experiment Time: 0.0005555555555555556h


,fov_object,fov,fov_x,fov_y,fov_z,fov_name,timestep,time,cell_line,channels,fname
0,<rtm_pymmcore.data_structures.Fov object at 0x...,0,0.00,0.0,0.0,0,0,0,FGFR_high,"({'name': 'DAPI', 'exposure': 150, 'group': 'C...",000_00000
2,<rtm_pymmcore.data_structures.Fov object at 0x...,1,20.01,0.0,0.0,1,0,0,FGFR_high,"({'name': 'DAPI', 'exposure': 150, 'group': 'C...",001_00000
1,<rtm_pymmcore.data_structures.Fov object at 0x...,0,0.00,0.0,0.0,0,1,2,FGFR_high,"({'name': 'DAPI', 'exposure': 150, 'group': 'C...",000_00001
3,<rtm_pymmcore.data_structures.Fov object at 0x...,1,20.01,0.0,0.0,1,1,2,FGFR_high,"({'name': 'DAPI', 'exposure': 150, 'group': 'C...",001_00001


### Run experiment

In [6]:
pymmcore_plus.configure_logging(stderr_level="WARNING")
for _ in range(0, SLEEP_BEFORE_EXPERIMENT_START_in_H * 3600):
    time.sleep(1)
from rtm_pymmcore.controller import ControllerSimulated, Analyzer
from rtm_pymmcore.dmd import DMD
from queue import Queue

try:
    mm_wdg._core_link.cleanup()
except:
    pass


analyzer = Analyzer(pipeline)
queue = Queue()
controller = ControllerSimulated(
    analyzer,
    mmc,
    queue,
    project_path=path_with_old_data_for_simulation,
)
controller.run(df_acquire)
print("Experiment finished")

fovs_i_list = os.listdir(os.path.join(path, "tracks"))
fovs_i_list.sort()
dfs = []
for fov_i in fovs_i_list:
    track_file = os.path.join(path, "tracks", fov_i)
    df = pd.read_parquet(track_file)
    dfs.append(df)
pd.concat(dfs).to_parquet(os.path.join(path, "exp_data.parquet"))

functional.py (238): The structure of `inputs` doesn't match the expected structure.
Expected: ['input']
Received: inputs=Tensor(shape=(1, 1024, 1024, 1))


Experiment finished
